In [1]:
#import necessary libraries
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Conv2D,MaxPooling2D,Conv1D,MaxPooling1D
from keras.layers import LSTM, SimpleRNN

In [3]:
import numpy as np
a = open("data_shuffle_input.csv")
net_data_input = np.genfromtxt(a, delimiter=",")
b = open("data_shuffle_target.csv")
net_data_target = np.genfromtxt(b, delimiter=",")
c = open("mydata_shuffle_input.csv")
net_mydata_input = np.genfromtxt(c, delimiter=",")
d = open("mydata_shuffle_target.csv")
net_mydata_target = np.genfromtxt(d, delimiter=",")

In [4]:
import random
from sklearn.model_selection import cross_val_score

net_data_input = net_data_input.reshape(net_mydata_input.shape[0],28,28,1)
net_mydata_input = net_mydata_input.reshape(net_data_input.shape[0],1,784)
net_mydata_input = net_mydata_input / 255

In [5]:
acc_per_fold = []
loss_per_fold = []


In [6]:
num = []
for i in range(len(net_data_target)):
    if net_data_target[i][0] == 1:
        num.append(0)
    if net_data_target[i][1] == 1:
        num.append(1)
    if net_data_target[i][2] == 1:
        num.append(2)
    if net_data_target[i][3] == 1:
        num.append(3)
    if net_data_target[i][4] == 1:
        num.append(4)
    if net_data_target[i][5] == 1:
        num.append(5)
    if net_data_target[i][6] == 1:
        num.append(6)
    if net_data_target[i][7] == 1:
        num.append(7)
    if net_data_target[i][8] == 1:
        num.append(8)
    if net_data_target[i][9] == 1:
        num.append(9)
num = np.asarray(num)        
print(type(num))


<class 'numpy.ndarray'>


In [7]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5)
acc_per_fold = []
acc = []
loss = []
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(net_data_input, net_data_target, net_mydata_input):

    # Define the model architecture for net mydata
    rnn_model_data = Sequential()
    #rnn_model.add(Flatten())
    rnn_model_data.add(LSTM(256,input_shape=net_mydata_input[0].shape,activation="tanh"))
    #rnn_model.add(SimpleRNN(128,input_shape=xtrain[0].shape,return_sequences=True))
    #rnn_model.add(SimpleRNN(128))
    rnn_model_data.add(Dense(units=10,activation="softmax"))

  # Compile the model
    rnn_model_data.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} data')

  # Fit data to model
    history_mydata = rnn_model_data.fit(net_mydata_input[train], net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)

    
    predictions_mydata_train = rnn_model_data.predict(net_mydata_input[train])
    classes_mydata_train = np.argmax(predictions_mydata_train, axis = 1)
    classes_mydata_train = tf.keras.utils.to_categorical(classes_mydata_train,10)
    
    predictions_mydata_test = rnn_model_data.predict(net_mydata_input[test])
    classes_mydata_test = np.argmax(predictions_mydata_test, axis = 1)
    classes_mydata_test = tf.keras.utils.to_categorical(classes_mydata_test,10)
    
    
    # Define the model architecture for net data
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='tanh', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
     # Generate a print
    # Fit data to model
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} mydata')
    history_data = model.fit(net_data_input[train], net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)

    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    predictions_data_train = model.predict(net_data_input[train])
    classes_data_train = np.argmax(predictions_data_train, axis = 1)
    classes_data_train = tf.keras.utils.to_categorical(classes_data_train,10)

    predictions_data_test = model.predict(net_data_input[test])
    classes_data_test = np.argmax(predictions_data_test, axis = 1)
    classes_data_test = tf.keras.utils.to_categorical(classes_data_test,10)
    
    #make train list input
    joined_list_train = []
    joined_list_test = []

    for i in range(800):
        b = [*classes_data_train[i], *classes_mydata_train[i]]
        joined_list_train.append(b)
        
    for j in range(200):
        a = [*classes_data_test[j], *classes_mydata_test[j]]
        joined_list_test.append(a)
    
    
    joined_list_train = np.asarray(joined_list_train)
    joined_list_test = np.asarray(joined_list_test)
    
    joined_list_train = joined_list_train.reshape(joined_list_train.shape[0],20,1)

    joined_list_test = joined_list_test.reshape(joined_list_test.shape[0],20,1)
    
  # Define the model architecture for net data
    print(joined_list_train[0].shape)
    rnn_model_outputs = Sequential()
    #rnn_model.add(Flatten())
    rnn_model_outputs.add(LSTM(256,input_shape=joined_list_train[0].shape,activation="tanh"))
    #rnn_model.add(SimpleRNN(128,input_shape=xtrain[0].shape,return_sequences=True))
    #rnn_model.add(SimpleRNN(128))
    rnn_model_outputs.add(Dense(units=10,activation="softmax"))

  # Compile the model
    rnn_model_outputs.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} outputs')

  # Fit data to model
    history = rnn_model_outputs.fit(joined_list_train, net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)
    scores = rnn_model_outputs.evaluate(joined_list_test, net_data_target[test], verbose=0)
    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    predictions = rnn_model_outputs.predict(joined_list_test)
    classes = np.argmax(predictions, axis = 1)
    print(num[test])
    acc_per_fold.append(accuracy_score(num[test],classes))
    
    print('accuracy is ' + str(accuracy_score(num[test],classes)))
    
  # Generate generalization metrics
    acc.append(history.history['acc'])
    loss.append(history.history['loss'])
    

  # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 data
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 0s 395us/sample - loss: 2.2926 - acc: 0.2562
Epoch 2/50
800/800 [==============================] - 0s 98us/sample - loss: 2.2589 - acc: 0.4812
Epoch 3/50
800/800 [==============================] - 0s 103us/sample - loss: 2.2230 - acc: 0.4888
Epoch 4/50
800/800 [==============================] - 0s 98us/sample - loss: 2.1806 - acc: 0.5512
Epoch 5/50
800/800 [==============================] - 0s 113us/sample - loss: 2.1285 - acc: 0.5775
Epoch 6/50
800/800 [==============================] - 0s 127us/sample - loss: 2.0673 - acc: 0.5938
Epoch 7/50
800/800 [==============================] - 0s 123us/sample - loss: 1.9936 - acc: 0.6025
Epoch 8/50
800/800 [==============================] - 0s 104us/sample - loss: 1.9075 - acc: 0.6475
Epoch 9/50
800/800 [==============================] - 0s 135us/sample - loss: 1.8120 - acc:

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


------------------------------------------------------------------------
Training for fold 1 mydata
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 0s 310us/sample - loss: 1.8773 - acc: 0.4313
Epoch 2/50
800/800 [==============================] - 0s 243us/sample - loss: 0.4872 - acc: 0.8575
Epoch 3/50
800/800 [==============================] - 0s 249us/sample - loss: 0.2846 - acc: 0.9225
Epoch 4/50
800/800 [==============================] - 0s 247us/sample - loss: 0.1872 - acc: 0.9563
Epoch 5/50
800/800 [==============================] - 0s 247us/sample - loss: 0.1130 - acc: 0.9800
Epoch 6/50
800/800 [==============================] - 0s 241us/sample - loss: 0.0736 - acc: 0.9875
Epoch 7/50
800/800 [==============================] - 0s 238us/sample - loss: 0.0573 - acc: 0.9950
Epoch 8/50
800/800 [==============================] - 0s 245us/sample - loss: 0.0398 - acc: 0.9975
Epoch 9/50
800/800 [==============================] - 0s 248us/sample - loss: 0.0317 - 

800/800 [==============================] - 0s 562us/sample - loss: 0.0171 - acc: 1.0000
Epoch 31/50
800/800 [==============================] - 0s 557us/sample - loss: 0.0158 - acc: 1.0000
Epoch 32/50
800/800 [==============================] - 0s 593us/sample - loss: 0.0147 - acc: 1.0000
Epoch 33/50
800/800 [==============================] - 0s 615us/sample - loss: 0.0137 - acc: 1.0000
Epoch 34/50
800/800 [==============================] - 0s 585us/sample - loss: 0.0128 - acc: 1.0000
Epoch 35/50
800/800 [==============================] - 0s 569us/sample - loss: 0.0120 - acc: 1.0000
Epoch 36/50
800/800 [==============================] - 1s 659us/sample - loss: 0.0113 - acc: 1.0000
Epoch 37/50
800/800 [==============================] - 0s 596us/sample - loss: 0.0107 - acc: 1.0000
Epoch 38/50
800/800 [==============================] - 0s 601us/sample - loss: 0.0101 - acc: 1.0000
Epoch 39/50
800/800 [==============================] - 0s 614us/sample - loss: 0.0096 - acc: 1.0000
Epoch 40/50


D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


...

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


------------------------------------------------------------------------
Training for fold 5 mydata
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 0s 376us/sample - loss: 2.2459 - acc: 0.4000
Epoch 2/50
800/800 [==============================] - 0s 258us/sample - loss: 0.6975 - acc: 0.8000
Epoch 3/50
800/800 [==============================] - 0s 258us/sample - loss: 0.3905 - acc: 0.8875
Epoch 4/50
800/800 [==============================] - 0s 259us/sample - loss: 0.2468 - acc: 0.9488
Epoch 5/50
800/800 [==============================] - 0s 270us/sample - loss: 0.1651 - acc: 0.9650
Epoch 6/50
800/800 [==============================] - 0s 238us/sample - loss: 0.1156 - acc: 0.9825
Epoch 7/50
800/800 [==============================] - 0s 230us/sample - loss: 0.0801 - acc: 0.9925
Epoch 8/50
800/800 [==============================] - 0s 231us/sample - loss: 0.0596 - acc: 0.9925
Epoch 9/50
800/800 [==============================] - 0s 231us/sample - loss: 0.0448 - 

800/800 [==============================] - 1s 721us/sample - loss: 0.0299 - acc: 1.0000
Epoch 31/50
800/800 [==============================] - 1s 717us/sample - loss: 0.0259 - acc: 1.0000
Epoch 32/50
800/800 [==============================] - 1s 727us/sample - loss: 0.0230 - acc: 1.0000
Epoch 33/50
800/800 [==============================] - 1s 722us/sample - loss: 0.0208 - acc: 1.0000
Epoch 34/50
800/800 [==============================] - 1s 722us/sample - loss: 0.0190 - acc: 1.0000
Epoch 35/50
800/800 [==============================] - 1s 733us/sample - loss: 0.0175 - acc: 1.0000
Epoch 36/50
800/800 [==============================] - 1s 712us/sample - loss: 0.0162 - acc: 1.0000
Epoch 37/50
800/800 [==============================] - 1s 701us/sample - loss: 0.0151 - acc: 1.0000
Epoch 38/50
800/800 [==============================] - 1s 729us/sample - loss: 0.0142 - acc: 1.0000
Epoch 39/50
800/800 [==============================] - 1s 690us/sample - loss: 0.0133 - acc: 1.0000
Epoch 40/50


D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[6 6 6 1 8 0 6 9 9 2 0 3 6 7 1 3 3 5 8 0 5 7 2 3 9 0 5 7 2 5 2 3 8 5 8 0 8
 9 8 7 9 8 7 5 0 5 1 7 3 1 6 3 6 5 6 9 2 5 1 3 6 2 9 1 4 8 7 8 0 5 2 7 4 0
 2 9 0 2 2 7 5 8 8 2 2 9 6 9 4 2 0 0 5 7 3 9 1 8 5 1 1 9 0 4 2 0 1 4 0 1 2
 2 8 0 0 1 6 5 4 4 4 8 1 5 8 8 1 4 8 8 3 9 0 7 4 8 4 2 2 5 8 1 3 9 1 0 3 0
 8 3 4 9 2 4 0 6 8 0 2 9 9 5 7 1 7 2 3 5 7 5 1 8 5 0 1 2 1 5 1 4 8 4 0 2 9
 0 0 1 6 0 6 8 5 8 3 4 8 6 3 5]
accuracy is 0.9


In [8]:

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 1.0699543434567749 - Accuracy: 0.91%
------------------------------------------------------------------------
> Fold 2 - Loss: 1.4709418201446534 - Accuracy: 0.88%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.9999448609352112 - Accuracy: 0.895%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.8306843361258507 - Accuracy: 0.92%
------------------------------------------------------------------------
> Fold 5 - Loss: 1.2324671506881715 - Accuracy: 0.9%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 0.901 (+- 0.013564659966250548)
> Loss: 1.1207985022701323
------------------------------------------------------------------------


In [9]:
np.savetxt('CNN(MN)_outputs_cross_validation_acc.csv',acc, delimiter=',')

In [10]:
np.savetxt('CNN(MN)_outputs_cross_validation_loss.csv',loss, delimiter=',')